In [45]:
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [46]:
def compare_columns(df: pd.DataFrame, coluna1: str, coluna2: str, coluna3: str, coluna4: str) -> list:
    """
    Compara duas colunas de um DataFrame e, com base no resultado, preenche uma nova coluna com os valores de uma terceira 
    coluna ou de uma quarta coluna do DataFrame. Retorna o DataFrame modificado.

    Args:
        df (pd.DataFrame): o DataFrame contendo as colunas a serem comparadas e a coluna a ser preenchida
        coluna1 (str): o nome da primeira coluna a ser comparada
        coluna2 (str): o nome da segunda coluna a ser comparada
        coluna3 (str): o nome da coluna a ser utilizada como valor se a primeira coluna for maior que a segunda
        coluna4 (str): o nome da coluna a ser utilizada como valor se a segunda coluna for maior que a primeira
        
    Returns:
        pd.DataFrame: o DataFrame modificado, com uma nova coluna preenchida com os valores da coluna3 ou da coluna4
        com base no resultado da comparação entre coluna1 e coluna2.
    """
    df["winner"] = [row[coluna3] if row[coluna1] > row[coluna2] else 'Empate' if row[coluna1] == row[coluna2] else row[coluna4] for i, row in df.iterrows()]
    
    return df

In [47]:
def filter_teams(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Filtra as linhas de um dataframe que contenham times que aparecem mais de 4 vezes no dataframe.

    Args:
        dataframe (pandas.DataFrame): O dataframe a ser filtrado.

    Returns:
        pandas.DataFrame: O dataframe filtrado.
    """
    team_counts = dataframe.groupby('winner').size()

    filtered_teams = team_counts[team_counts > 4].index.tolist()

    filtered_dataframe = dataframe[dataframe['winner'].isin(filtered_teams)]

    return filtered_dataframe

In [48]:
df = pd.read_csv('results.csv')

In [49]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [50]:
# Faz uma cópia do dataframe
dados = df.copy()

# Recebe o retorno da função compare_columns
dados = compare_columns(dados, 'home_score', 'away_score', 'home_team', 'away_team')

# Altera o tipo da coluna date para tipo data e cria uma coluna de ano
dados["date"] = pd.to_datetime(dados["date"], format='%Y-%m-%d')
dados["year"] = dados["date"].dt.year

# Cria um DataFrame agrupado por winner, year e quantidade de winner por ano
dados = dados.groupby(['winner', 'year']).size().reset_index(name="qtd_win").sort_values(by="qtd_win", ascending=False)

# Remove os registros com winner igual a empate
dados = dados[dados['winner'] != 'Empate']

# Recebe o retorno da função filter_teams
dados = filter_teams(dados)

## LinearRegression

In [52]:
# Cria uma lista com os nomes dos times únicos na base de dados
winners_unicos = dados["winner"].unique()

# Cria um dicionário para armazenar as previsões de cada time
previsoes = {}

# Cria listas vazias para armazenar os dados de entrada e saída
X = []
y = []

# Itera sobre cada time único na base de dados
for winner in winners_unicos:
    # Filtra os dados para obter apenas as linhas que correspondem a este time
    dados_winner = dados[dados["winner"] == winner]
    
    # Separa os dados de entrada (anos) e saída (jogos_ganhos)
    X_winner = dados_winner[["year"]]
    y_winner = dados_winner["qtd_win"]
    
    # Adiciona os dados de entrada e saída às listas globais
    X.extend(X_winner.values)
    y.extend(y_winner.values)
    
    # Separa os dados em treino e teste, utilizando 80% para treino e 20% para teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X_winner, y_winner, test_size=0.2, random_state=42)
    
    # Cria o modelo de regressão linear
    modelo = LinearRegression()
    
    # Treina o modelo com os dados de treino
    modelo.fit(X_treino, y_treino)
    
    # Faz a previsão para o ano de 2023 e armazena no dicionário de previsões
    previsao = modelo.predict([[2023]])
    previsoes[winner] = previsao[0]

# Cria um dataframe com as previsões para 2023
previsoes_df = pd.DataFrame.from_dict(previsoes, orient="index", columns=["qtd_win"])
previsoes_df.index.name = "winner"
previsoes_df["year"] = 2023
previsoes_df = previsoes_df.reset_index().reindex(columns=["year", "winner", "qtd_win"])

# Calcula o MSE e o MAE do modelo
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = LinearRegression()
modelo.fit(X_treino, y_treino)
y_pred = modelo.predict(X_teste)
mse = mean_squared_error(y_teste, y_pred)
mae = mean_absolute_error(y_teste, y_pred)

# Imprime o dataframe com as previsões e o MSE e MAE do modelo
print(previsoes_df.sort_values(by="qtd_win", ascending=False))
print(f"MSE do modelo: {mse:.2f}")
print(f"MAE do modelo: {mae:.2f}")
print(f"RMSE do modelo: {sqrt(mse):.2f}")

     year             winner    qtd_win
0    2023             Brazil  11.144355
5    2023             Mexico  10.327836
65   2023          German DR   9.701912
12   2023               Iran   9.326943
30   2023            Germany   8.854502
..    ...                ...        ...
229  2023        Timor-Leste   0.142857
187  2023     Cayman Islands   0.054895
84   2023    Northern Cyprus  -0.289256
239  2023              Frøya  -0.389831
198  2023  Western Australia  -6.452830

[247 rows x 3 columns]
MSE do modelo: 6.32
MAE do modelo: 1.91
RMSE do modelo: 2.51


## RandomForestRegressor

In [53]:
# Cria uma lista com os nomes dos times únicos na base de dados
winners_unicos = dados["winner"].unique()

# Cria um dicionário para armazenar as previsões de cada time
previsoes = {}

# Cria listas vazias para armazenar os dados de entrada e saída
X = []
y = []

# Itera sobre cada time único na base de dados
for winner in winners_unicos:
    # Filtra os dados para obter apenas as linhas que correspondem a este time
    dados_winner = dados[dados["winner"] == winner]
    
    # Separa os dados de entrada (anos) e saída (jogos_ganhos)
    X_winner = dados_winner[["year"]]
    y_winner = dados_winner["qtd_win"]
    
    # Adiciona os dados de entrada e saída às listas globais
    X.extend(X_winner.values)
    y.extend(y_winner.values)
    
    # Separa os dados em treino e teste, utilizando 80% para treino e 20% para teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X_winner, y_winner, test_size=0.2, random_state=42)
    
    # Cria o modelo de Random Forest
    modelo = RandomForestRegressor(n_estimators=50, random_state=42)
    
    # Treina o modelo com os dados de treino
    modelo.fit(X_treino, y_treino)
    
    # Faz a previsão para o ano de 2023 e armazena no dicionário de previsões
    previsao = modelo.predict([[2023]])
    previsoes[winner] = previsao[0]

# Cria um dataframe com as previsões para 2023
previsoes_df = pd.DataFrame.from_dict(previsoes, orient="index", columns=["qtd_win"])
previsoes_df.index.name = "winner"
previsoes_df["year"] = 2023
previsoes_df = previsoes_df.reset_index().reindex(columns=["year", "winner", "qtd_win"])

# Calcula o MSE e MAE do modelo
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestRegressor(n_estimators=50, random_state=42)
modelo.fit(X_treino, y_treino)
y_pred = modelo.predict(X_teste)
mse = mean_squared_error(y_teste, y_pred)
mae = mean_absolute_error(y_teste, y_pred)

# Imprime o dataframe com as previsões, o MSE e o MAE do modelo
print(previsoes_df.sort_values(by="qtd_win", ascending=False))
print(f"MSE do modelo: {mse:.2f}")
print(f"MAE do modelo: {mae:.2f}")
print(f"RMSE do modelo: {sqrt(mse):.2f}")

     year        winner  qtd_win
37   2023     Argentina    10.00
35   2023       Morocco    10.00
40   2023       Senegal     9.94
19   2023        France     9.34
22   2023       Algeria     9.22
..    ...           ...      ...
229  2023   Timor-Leste     1.00
228  2023       Somalia     1.00
227  2023  Sint Maarten     1.00
203  2023         Macau     1.00
246  2023  Saare County     1.00

[247 rows x 3 columns]
MSE do modelo: 6.29
MAE do modelo: 1.91
RMSE do modelo: 2.51


## MLPRegressor

In [57]:
# Cria uma lista com os nomes dos times únicos na base de dados
winners_unicos = dados["winner"].unique()

# Cria um dicionário para armazenar as previsões de cada time
previsoes = {}

# Cria listas vazias para armazenar os dados de entrada e saída
X = []
y = []

# Itera sobre cada time único na base de dados
for winner in winners_unicos:
    # Filtra os dados para obter apenas as linhas que correspondem a este time
    dados_winner = dados[dados["winner"] == winner]
    
    # Separa os dados de entrada (anos) e saída (jogos_ganhos)
    X_winner = dados_winner[["year"]]
    y_winner = dados_winner["qtd_win"]
    
    # Adiciona os dados de entrada e saída às listas globais
    X.extend(X_winner.values)
    y.extend(y_winner.values)
    
    # Separa os dados em treino e teste, utilizando 80% para treino e 20% para teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(X_winner, y_winner, test_size=0.2, random_state=42)
    
    # Cria o modelo de rede neural
    modelo = MLPRegressor(hidden_layer_sizes=(5, 15, 2, 30), activation='identity', solver='lbfgs', max_iter=500)
    
    # Treina o modelo com os dados de treino
    modelo.fit(X_treino, y_treino)
    
    # Faz a previsão para o ano de 2023 e armazena no dicionário de previsões
    previsao = modelo.predict([[2023]])
    previsoes[winner] = previsao[0]

# Cria um dataframe com as previsões para 2023
previsoes_df = pd.DataFrame.from_dict(previsoes, orient="index", columns=["qtd_win"])
previsoes_df.index.name = "winner"
previsoes_df["year"] = 2023
previsoes_df = previsoes_df.reset_index().reindex(columns=["year", "winner", "qtd_win"])

# Calcula o MSE e MAE do modelo
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = MLPRegressor(hidden_layer_sizes=(5, 15, 2, 30), activation='identity', solver='lbfgs', max_iter=500)
modelo.fit(X_treino, y_treino)
y_pred = modelo.predict(X_teste)
mse = mean_squared_error(y_teste, y_pred)
mae = mean_absolute_error(y_teste, y_pred)

# Imprime o dataframe com as previsões, o MSE e o MAE do modelo
print(previsoes_df.sort_values(by="qtd_win", ascending=False))
print(f"MSE do modelo: {mse:.2f}")
print(f"MAE do modelo: {mae:.2f}")
print(f"RMSE do modelo: {sqrt(mse):.2f}")

     year                        winner   qtd_win
30   2023                       Germany  8.854414
8    2023                         Spain  8.533725
37   2023                     Argentina  7.390738
4    2023                  Saudi Arabia  7.355753
0    2023                        Brazil  7.281563
..    ...                           ...       ...
230  2023  United States Virgin Islands  1.002761
245  2023                        Monaco  1.001071
198  2023             Western Australia  0.952313
140  2023                Basque Country  0.862107
207  2023                         Sápmi  0.833241

[247 rows x 3 columns]
MSE do modelo: 6.62
MAE do modelo: 1.98
RMSE do modelo: 2.57


In [30]:
dados[dados['winner'] == 'Morocco'].sort_values(by="year", ascending=False)

,winner,year,qtd_win
5900,Morocco,2022,11
5899,Morocco,2021,12
5898,Morocco,2020,1
5897,Morocco,2019,6
5896,Morocco,2018,11
...,...,...,...
5844,Morocco,1963,2
5843,Morocco,1962,1
5842,Morocco,1961,3
5841,Morocco,1960,1
